In [1]:
# Current as of June 5, 2020

! pip install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time
import pandas as pd
import sys


     |████████████████████████████████| 911kB 2.7MB/s 


In [12]:
! wget https://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_linux64.zip && unzip chromedriver_linux64

--2020-06-05 21:48:22--  https://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.217.204.128, 2607:f8b0:400c:c15::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5306126 (5.1M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   5.06M  --.-KB/s    in 0.04s   

2020-06-05 21:48:22 (137 MB/s) - ‘chromedriver_linux64.zip’ saved [5306126/5306126]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            


In [13]:
import os
#Mount the drive from Google to save the dataset
from google.colab import drive # this will be our driver
drive.mount('/gdrive')
root = '/gdrive/My Drive/' # if you want to operate on your Google Drive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
colab_path = '/gdrive/../content/'
chromedriver_path = '/gdrive/../content/chromedriver'

# Silently open chrome browser
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(executable_path=chromedriver_path,options=chrome_options)


# Get all relevant product links


In [0]:
from IPython.display import clear_output
clear_output()

# Set product category 
# In this format options are ['blush', 'bronzer', 'lipstick', 'lip-gloss', 
## 'lip-stain', 'lip-liner-lip-pencils']
category = 'lipstick'

# Base url to reach product lists
# 300 items per page is enough to capture all products
base_url = 'https://www.sephora.com/shop/{}?pageSize=300'.format(category) 

# Go to url
wd.get(base_url)

# Get total number of products
header = wd.find_element_by_css_selector("[data-at='number_of_products']")
no_products = int(header.text.split(' ')[0])

product_group = wd.find_elements_by_class_name("css-dkxsdo") # 12 products per group

# TODO: Implement wait function to make sure class names have refreshed upon
## scrolling.
# wait = WebDriverWait(wd, 1) # seconds
# wait.until(EC.presence_of_element_located((By.CLASS_NAME,"css-13ys60n")))


# wait.until(EC.visibility_of_element_located((By.CLASS_NAME,"css-ix8km1")))

# Instantiate empty array for eventual dataframe
df_product = []

# Loop through products on page and retrieve name and hyperlink
# Note: Product icon change class name upon scrolling into view
# We need to make sure that has happened before trying to retrieve the new class name
wait = WebDriverWait(wd, 5) # seconds
wait.until(EC.visibility_of_element_located((By.CLASS_NAME,"css-ix8km1")))

for ii, product in enumerate(product_grid):
    print(ii)
    product_info = product.find_element_by_class_name("css-ix8km1")
    product_name = product_info.get_attribute('aria-label')
    product_link = product_info.get_attribute('href')

    # add product to dataframe array
    df_product.append([product_name, product_link])

# Name columns for dataframe 
columns = ['product_name', 'product_link']

# Make dataframe
df = pd.DataFrame(df_product, columns=columns)

# Save dataframe
df.to_csv('data/{}.csv'.format(category), index=False)

## Loop through all products

In [0]:
# load category product file
df_all_category = pd.read_csv('data/' + category + '.csv')


for ii in range(3, len(df_all_category)):
    row = df_all_category.iloc[ii]
    product_name = row['product_name']
    product_link = row['product_link']
    product_id = 'P' + product_link.split(':p')[-1]
  
    # open url in instantiated browser
    wd.get(product_link)
    
    
    # retrieve product specs
    product_header = wd.find_element_by_tag_name("h1").text
    brand_name = product_header.split('\n')[0]
        
            
    no_of_reviews = int(driver.find_element_by_class_name("css-tc6qfq").text.split(' ')[-0])
    reviews_container = driver.find_element_by_id("ratings-reviews-container")
    reviews = reviews_container.find_elements_by_class_name("css-1ecc607")
    loaded_no_of_reviews = len(reviews)
    print(no_of_reviews, loaded_no_of_reviews)
    
    # close pop-up login window
    try:
        login_window = driver.find_element_by_class_name('css-fslzaf')
        login_window.click()
    except (NoSuchElementException, ElementNotVisibleException):
        pass
            
    if no_of_reviews <= 100:
        while loaded_no_of_reviews < no_of_reviews:
            more_reviews = driver.find_element_by_class_name("css-frqcui")
            more_reviews.click()

            # count number of visible reviews -> become new loaded reviews
            reviews = reviews_container.find_elements_by_class_name("css-1ecc607")
            loaded_no_of_reviews = len(reviews)
  
            
    elif no_of_reviews > 100:
        
        while loaded_no_of_reviews < 100:    
            # "Show 6 more reviews"
            more_reviews = driver.find_element_by_class_name("css-frqcui")
            more_reviews.click()

            # count number of visible reviews -> become new loaded reviews
            reviews = reviews_container.find_elements_by_class_name("css-1ecc607")
            loaded_no_of_reviews = len(reviews)
        
        
    # inspection of reviews in browser gives us desired class name to search for 
    reviews_container = driver.find_element_by_id("ratings-reviews-container")
    reviews = reviews_container.find_elements_by_class_name("css-1ecc607")



    columns = ['product_category', 'brand', 'product_name', 'user_id', 'BIP_level' ,'BIC_level', 
               'user_specs','rating', 'review_date','verified', 'review_color', 'review_color_img', \
               'review_title', 'review_content', 'no_helpful', 'no_not_helpful', 'recommendation']


    df_reviews = []

    for review in reviews:
        product_category = category
        brand = brand_name
        product_name = product_name

        reviewer_info = review.find_element_by_class_name("css-vq8ezb")

        user_id = ''
        try:
            user_id = reviewer_info.find_element_by_css_selector("[data-at='nickname']").text
        except NoSuchElementException:
            pass


        BIP_level = ''
        BIC_level = ''
        try:
            statuses = reviewer_info.find_elements_by_tag_name("img")
            if len(statuses) > 1:
                BIP_level = statuses[0].get_attribute("alt")
                BIC_level = statuses[1].get_attribute("alt")
        except NoSuchElementException:
            pass


        user_specs = ''
        try:
            user_specs = [spec.text for spec in reviewer_info.find_elements_by_class_name("css-10lg0rx")]

        except NoSuchElementException:
            pass


        rating = ''
        try:
            rating = review.find_element_by_class_name("css-3z5ot7").get_attribute("aria-label")
        except NoSuchElementException:
            pass

        review_date = ''
        try:
            review_date = review.find_element_by_css_selector("[data-at='time_posted']").text
        except NoSuchElementException:
            pass

        verified = ''
        try:
            if_verified = review.find_element_by_css_selector("[data-at='verified_purchase_badge']")
            verified = 'yes'
        except NoSuchElementException:
            verified= 'no'
            pass

        review_color = ''
        try:
            review_color = review.find_element_by_class_name("css-gjgyg3").text
        except NoSuchElementException:
            pass

        review_color_img = ''
        try:
            review_color_img_elem = review.find_element_by_css_selector("[role='presentation']")
            review_color_img = review_color_img_elem.get_attribute("src")

        except NoSuchElementException:
            pass

        review_title = ''
        try:
            review_title = review.find_element_by_class_name("css-85k18d").text
        except NoSuchElementException:
            pass  

        review_content = ''
        try:
            review_content = review.find_element_by_class_name("css-7rv8g1").text
        except NoSuchElementException:
            pass  


        no_helpful = 0
        no_not_helpful = 0
        try:
            helpful_buttons = review.find_elements_by_class_name("css-ji5a4n")
            no_not_helpful = helpful_buttons[0].text.split(' ')[-1]
            no_helpful = helpful_buttons[1].text.split(' ')[-1]
        except NoSuchElementException:
            pass

        recommendation = ''
        try: 

            recommendation = review.find_element_by_class_name("css-1nv53ng").text

        except NoSuchElementException:
            pass

        df_reviews.append(
                [category, brand, product_name, user_id, BIP_level, BIC_level, user_specs,
                 rating, review_date, verified, review_color, review_color_img, review_title, review_content, 
                 no_helpful, no_not_helpful, recommendation])
    
    print('Done gathering info on: {}'.format(product_name))
    df = pd.DataFrame(df_reviews, columns=columns)
    df['no_helpful'] = df['no_helpful'].str.replace(r"\(","")
    df['no_helpful'] = df['no_helpful'].str.replace(r"\)","")
    df['no_not_helpful'] = df['no_not_helpful'].str.replace(r"\(","")
    df['no_not_helpful'] = df['no_not_helpful'].str.replace(r"\)","")

    df.to_csv('data/' + category + "_" + str(product_id) + '.csv', index=False)
    
    driver.close()